#UE03 - SPARQL Query

Please complete the 10 tasks in the `2. SPARQL` sheet of `SemAI.jar` first, and then transfer the task descriptions and your solutiosn in executable form to this notebook.

## Preparation

Reuse imports and functions from https://github.com/jku-win-dke/SemAI/blob/main/V02_SPARQL.ipynb and load the solar system graph. Query the solar system graph to check that everything works fine. 

In [ ]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def sparql_construct(graph, query):
  result_graph = Graph(namespace_manager = g.namespace_manager)  # create a Graph object that reuses the namespace prefixes of the original graph
  result_graph += graph.query(query)                             # execute the construct query against the original graph and add the resulting graph to the new one
  return result_graph

def sparql_ask(graph, query):
  return bool(graph.query(query))      # an ASK query has a boolean result, which should be returned as such

# Load solar system graph
g = Graph()
g.parse("https://raw.githubusercontent.com/jku-win-dke/SemAI/main/data/solarsystem.ttl",format="turtle")

# Query solar system graph (to check that everything works fine)
df = sparql_select(g,"""
  SELECT ?planet ?apoapsis ?apoapsis_uom
  WHERE { 
    ?planet rdf:type dbo:Planet . 
    OPTIONAL { ?planet v:apoapsis [rdf:value ?apoapsis ; v:uom ?apoapsis_uom ].  }
  }
""")
df


## Task 1 (1 pt)

Geben Sie alle Zwergplaneten (Instanzen der Klasse dbo:DwarfPlanet) aus und falls vorhanden deren genaue Entsprechung (skos:exactMatch). Ordnen Sie das Ergebnis aufsteigend nach den URIs der Zwergplaneten.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_select(g,"""
SELECT ?d ?match
WHERE { 
	?d rdf:type dbo:DwarfPlanet .
	OPTIONAL{ ?d skos:exactMatch ?match. }
}
ORDER BY asc(?d)
""")
df

## Task 2 (1 pt)

Ermitteln Sie alle Sterne, ihr exactMatch, und ihre Masse (geben Sie den Wert und die Maßeinheit aus). Ordnen Sie das Ergebnis nach der URI der Sterne

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_select(g,"""
SELECT ?star ?match ?massVal ?massUoM
WHERE { 
	?star rdf:type dbo:Star.
	?star skos:exactMatch ?match.
        ?star v:mass [rdf:value ?massVal].
        ?star v:mass [v:uom ?massUoM].
}
ORDER BY asc(?star)
""")
df

## Task 3 (1 pt)

Die Planeten unseres Sonnensystems und ihre jeweilige Anzahl an Monden. Unterscheiden Sie die im RDF-Graph beschriebenen Monde und die im RDF-Graph erfasste Anzahl von Monden. Sortieren Sie nach den Planeten.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_select(g,"""
SELECT ?planet ?assertedNo (count(?moon) as ?noOfDescribedMoons )
WHERE { 
	?planet rdf:type dbo:Planet.
	?planet v:nrOfMoons ?assertedNo.
        OPTIONAL {?moon v:orbits ?planet.}
}
GROUP BY ?planet ?assertedNo
ORDER BY asc(?planet )
""")
df

## Task 4 (1 pt)

Ermitteln sie für die Planeten in unserem Sonnensystem die durchschnittliche Anzahl an Monden (die auch im RDF-Graph beschrieben sind) pro Planet.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_select(g,"""
SELECT (avg(?moonCount) as ?avgNoOfDescribedMoons ) WHERE{
SELECT (count(?moon) as ?moonCount) 
WHERE { 
	?planet rdf:type dbo:Planet.
	?planet v:nrOfMoons ?assertedNo.
        OPTIONAL {?moon v:orbits ?planet.}
}
GROUP BY ?planet ?assertedNo
}
""")
df

## Task 5 (1 pt)

Geben Sie die im RDF-Graph verwendeten Klassen und ihre Anzahl an Instanzen aus. Geben Sie nur Klassen mit mindestens 2 Instanzen aus. Ordnen Sie die Ausgabe nach der URI der Klassen.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_select(g,"""
SELECT ?class ( COUNT(?s) as ?noOfInstances)
WHERE { ?s rdf:type ?class  }
GROUP BY ?class
HAVING (COUNT(?s) > 2)
ORDER BY ?class
""")
df

## Task 6 (1 pt)

Geben Sie die im RDF-Graph verwendeten Properties und ihre Häufigkeit je Klasse aus. Gezählt werden sollen die Instanzen der Klasse, die die Property ausprägen. Geben Sie nur Properties zu Klassen aus, die von mindestens 7 Instanzen dieser Klasse ausgeprägt werden. Ordnen Sie die Ausgabe zuerst nach der Klasse und dann nach der Property.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_select(g,"""
SELECT ?class ?prop ( COUNT(distinct ?s) as ?noOfInstances )
WHERE { 
?s rdf:type ?class.
?s ?prop ?o.
}
GROUP BY ?class ?prop
HAVING (COUNT(distinct ?s) >= 7)
ORDER BY ?class ?prop
""")
df

## Task 7 (1 pt)

Ermitteln Sie alle Sterne, sowie alle Monde. Zu jedem Mond ermitteln Sie auch den Planet, den dieser Mond umrundet. Ordnen Sie die Ausgabe nach der URI der Sterne und Monde.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_select(g,"""
SELECT ?x ?y
WHERE { 
{
?x rdf:type dbo:Satellite .
?x v:orbits ?y.

} 
UNION { 
?x rdf:type dbo:Star.
}
}
ORDER BY ?x
""")
df

## Task 8 (1 pt)

Erzeugen sie mittels einer Construct-Query den angezeigten RDF-GRAPH (Expected Result). Reihenfolge ist unerheblich.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_construct(g,"""
CONSTRUCT { 
?x  v:wirdUmrundetVon ?sub .
?x_hack rdf:type v:Himmelskoerper. 
}
WHERE { 
?x (v:orbits)* :Sun . 
?x rdf:type ?nodp . 
FILTER(?nodp  NOT IN (dbo:DwarfPlanet)) . 
OPTIONAL {?sub v:orbits ?x .
?sub rdf:type ?nodp1 .
FILTER(?nodp1 NOT IN (dbo:DwarfPlanet))
}.
?x_hack (v:orbits)+ :Sun .
?x_hack rdf:type ?b .
FILTER(?b NOT IN (dbo:DwarfPlanet))
}
""")
print(df.serialize(format="turtle"))

## Task 9 (1 pt)

Gibt es einen Stern im RDF-Graph, der massereicher als die Sonne ist.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_ask(g,"""
ASK{ 
?x v:mass  [rdf:value ?v; v:uom ?u] .
BIND (IF( ?u = v:SolarMass, 1988435000000000000000000000000.0 , 1)  as ?conversion ) .
BIND (( ?v * ?conversion ) as ?m  ) .
FILTER(?x NOT IN (:Sun) ) .
?sun v:mass [rdf:value ?s ] . 
FILTER(?sun IN (:Sun)) .
FILTER ( ?m > ?s ).
}
""")
df

## Task 10 (1 pt)

Geben Sie eine Beschreibung der Resourcen aus, die einen Durchmesser zwischen 20.000 km und 30.000 km haben.

In [ ]:
# TODO: include and execute the query, display the result
df = sparql_construct(g,"""
DESCRIBE ?s
WHERE { ?s v:radius [rdf:value ?o; v:uom ?u ] .
FILTER ( ?o > 20000 && ?o < 30000)
}

""")
print(df.serialize(format="turtle"))